In [2]:
import os, sys
import numpy as np
env_var = os.environ

def ls_level(folder, nlevels):
    for ii in range(nlevels):
        folder = os.path.dirname(folder)
    folder += "/"
    return folder

path_program = ls_level(os.getcwd(), 1)
print(path_program)
sys.path.append(path_program)
os.environ["LACE_REPO"] = ls_level(os.getcwd(), 2) + 'LaCE'
os.environ["LACE_MANAGER_REPO"] = ls_level(os.getcwd(), 2) + 'LaCE_manager'

# os.environ["LACE_REPO"] = "/nfs/pic.es/user/l/lcabayol/DESI/LaCE"
# os.environ["LACE_MANAGER_REPO"] = "/nfs/pic.es/user/l/lcabayol/DESI/LaCE_manager"

/home/jchaves/Proyectos/projects/lya/LaCE/


In [3]:
from lace.emulator.nn_architecture import MDNemulator_polyfit
from lace.emulator.nn_emulator import NNEmulator
from lace.emulator.gp_emulator import GPEmulator

from lace.emulator.emulator import P1D_emulator

from lace.emulator import pnd_archive

In [4]:
# our modules
from lace.cosmo import fit_linP
from lace.emulator import poly_p1d
from lace.archive import pnd_archive
from lace.archive import interface_archive

In [5]:
emuparams = ['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']

# CREATE TRAINING AND TESTING ARCHIVE

In [6]:
archive = pnd_archive.archivePND(sim_suite="Cabayol23")
archive.get_training_data()
len(archive.training_data)

9900

In [7]:
archive_test = pnd_archive.archivePND(sim_suite="Cabayol23", pick_sim='central')
archive_test.get_testing_data()
len(archive_test.testing_data)

11

## TRAIN NEURAL NETWORK EMULATOR  

By default, the NNemulator is trained with the following parameters:ç
- zmax=4.5
- Nsim=30 
- nepochs=100
- step_size=75
- kmax_Mpc=4
- ndeg=5. For the extended version (kmax_Mpc=8), set to 7
- postprocessing='768'. To train with the 1D post-processing change to '500'

- initial_weights=True. Always starts from the same iniial parameters. These were selected randomly. Set to False if random initialization is required.

In [8]:
emulator = NNEmulator(archive,emuparams,ndeg=5, save_path=None, nepochs=1)

start the training of the emulator
Training network on 9900
Emualtor trained in 0.42965149879455566 seconds


## LOAD A NEURAL NETWORK EMULATOR

In [9]:
emulator = NNEmulator(archive,emuparams,ndeg=5, save_path=None, nepochs=1, model_path='NNmodels/NNEmulator_LaCEHC.pt', train=False)

Model loaded. No training needed


## TEST NEURAL NETWORK EMULATOR

In [10]:
p1d, p1derr=emulator.emulate_p1d_Mpc(archive_test.testing_data[0])

# Gaussian Process passing an archive

- It is trained on 330 P1D. By default, it does not use optical-depth rescalings
- The default options for $k$, $z$, and 'deg' are the same as for the NN emulator

In [11]:
archive = pnd_archive.archivePND(sim_suite="Pedersen21")
archive.get_training_data()
len(archive.training_data)

330

In [12]:
emulator = GPEmulator(passarchive=archive)

 /home/jchaves/miniconda3/envs/general/lib/python3.9/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 330 points
GPs optimised in 0.43 seconds


In [13]:
kMpc = archive_test.testing_data[0]['k_Mpc'][1:43]

In [14]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

# Gaussian Process without passing an archive

In [16]:
emulator = GPEmulator()

Training GP on 330 points
GPs optimised in 0.54 seconds


In [17]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

# MODULE COMBIING NN AND GP EMULATORS

def P1D_emulator(archive=None, emu_algorithm=None, archive_label = 'Gadget', emulator_label=None, zmax=4.5, kmax_Mpc=4, ndeg=5, train=True, emu_type='polyfit', model_path=None, save_path=None, nepochs_nn=100):
    
    
    
    if emulator_label=='Pedersen21':
        print('Select emulator used in Pedersen et al. 2021')
        emuparams = ['Delta2_p', 'n_p', 'mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
        archive = pnd_archive.archivePND(sim_suite="Pedersen21")
        archive.get_training_data()
        
        emulator = GPEmulator(passarchive=archive)
        return emulator

    elif emulator_label=='Cabayol23':
        print('Select emulator used in Cabayol-Garcia et al. 2023')
        emuparams = ['Delta2_p', 'n_p', 'mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
        if train==True:
            print('Train emulator')
            archive = pnd_archive.archivePND(sim_suite="Cabayol23")
            archive.get_training_data()

            emulator = NNEmulator(archive,emuparams, nepochs=nepochs_nn, step_size=75, kmax_Mpc=4, ndeg=5, Nsim=30, train=True)
        if train==False:
            if model_path==None:
                raise ValueError('if train==False, a model path must be provided')
            else:
                print('Load pre-trained emulator')
                archive = pnd_archive.archivePND(sim_suite="Cabayol23")
                archive.get_training_data()
                emulator = NNEmulator(archive,emuparams, train=False,model_path=model_path)
                
    if emulator_label==None:
        print('Select custom emulator')
        if (archive==None)|(emu_algorithm==None):
            raise ValueError('If an emulator label is not provided, the archive and emulator algorithm are required')

        if (emu_algorithm=='NN')&(archive_label=='Gadget'):
            emuparams = ['Delta2_p', 'n_p', 'mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
            emulator = NNEmulator(archive,emuparams,ndeg=ndeg, zmax=zmax, kmax_Mpc=kmax_Mpc,save_path=save_path )

        if (emu_algorithm=='NN')&(archive_label=='Nyx'):
            emuparams = ['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'lambda_P']
            raise Exception('Work in progress')

        if (emu_algorithm=='GP')&(archive_label=='Gadget'):
            emuparams = ['Delta2_p', 'n_p', 'mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
            emulator = GPEmulator(passarchive=archive)

                
    return emulator

## LOAD EMULATOR IN PEDERSEN+21

In [18]:
emulator = P1D_emulator(emulator_label='Pedersen21')

Select emulator used in Pedersen et al. 2021
Training GP on 330 points
GPs optimised in 0.50 seconds


## LOAD EMULATOR IN CABAYOL+23

### train the emulator

In [19]:
emulator = P1D_emulator(emulator_label='Cabayol23',nepochs_nn=1)

Select emulator used in Cabayol-Garcia et al. 2023
Train emulator
start the training of the emulator
Training network on 9900
Emualtor trained in 0.4126107692718506 seconds


### Load trained emulator

In [20]:
emulator = P1D_emulator(emulator_label='Cabayol23', train=False, model_path='NNmodels/NNEmulator_LaCEHC.pt')

Select emulator used in Cabayol-Garcia et al. 2023
Load pre-trained emulator
Model loaded. No training needed


## LOAD CUSTOM EMULATOR

In [21]:
# This is supposed to fail
emulator = P1D_emulator()

Select custom emulator


ValueError: If an emulator label is not provided, the archive and emulator type are required

In [22]:
emulator = P1D_emulator(archive=archive, emu_algorithm='NN')

Select custom emulator
start the training of the emulator
Training network on 330
Emualtor trained in 1.5460195541381836 seconds


In [23]:
emulator = P1D_emulator(archive=archive, emu_algorithm='GP')

Select custom emulator
Training GP on 330 points
GPs optimised in 0.54 seconds
